# Gestion des données tabulaires

Les données tabulaires (ensemble de données regroupées par colonnes et par lignes) sont universelles et omniprésentes. On les retrouve aussi bien dans nos listes de courses, nos listes de taches, la gestion comptable et le suivi des processus, les formats d'échange. 

Des outils dédiés leurs sont consacrés (les tableurs) et elles sont à la base également de solutions élaborées comme par exemple pour le stockage (les 'tables' des bases de données relationnelles) et pour le traitement (les 'dataFrame' Pandas) de gros volumes de données.

Pourtant, malgré cette importante popularité, il n'existe que très peu d'outils permettant de comprendre, d'analyser et d'exploiter la structuration de ce type de données.

L'objet de cet article est donc de présenter les améliorations importantes qu'apporte une meilleure gestion de nos données tabulaires.

L'article est découpé en deux parties:
- une [première partie](#1---Structures-tabulaires) explicative accompagnée d'un exemple,
- une [deuxième partie](#2---Exemple-d'outil) justificative et de démonstration s'appuyant sur un outil dédié

Les liens présents dans les deux parties facilitent l'interactivité entre celles-ci.

Cet article est également consultable sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/documentation) (meilleure lecture des tableaux).

# 1 - Structures tabulaires
Cette première partie précise ce que sont les structures tabulaires et comment elles peuvent être exploitées (indépenamment des données qu'elles contiennent).   
    
Elle présente successivement:
- la nature des structures tabulaires,
- l'analyse qui permet de révéler ces structures 
    - au niveau des relations entre champ
    - au niveau de la structure complète,
- l'exploitation de la structure pour :
    - préparer les données pour un usage par des outils de traitement
    - générer des formats d'échange optimisés (gain d'un facteur 5 à 10 par rapport à un fichier csv)
    - garantir l'intégrité des données
    - disposer de processus de réutilisation qui respectent les données d'origine
    - proposer des moyens d'accès optimisés
    - fournir des indicateurs opérationnels de qualité des données


## Que trouve-t-on dans un tableau ?

La première structure identifiable est évidente : les lignes et les colonnes !    
Néanmoins, ces deux notions ne sont pas équivalentes, les colonnes (ou champs) représentent la 'sémantique' des données et les lignes représentent les objets rangés suivant cette structuration définie par les colonnes.

Si l'on observe maintenant comment sont utilisés les tableaux, on peut identifier trois usages principaux :
- la classification : Il s'agit de regrouper les objets par catégories afin de pouvoir par exemple en faire une exploitation statistique,
- le croisement : Il consiste à représenter toutes les combinaisons entre deux paramètres comme par exemple dans les représentations matricielles
- la caractérisation : Elle correspond à la documentation de propriétés définies.

> *Prenons un exemple :*
>
>|id|produit|aliment	|quantité	|prix|validité|disponibilité|
|:------:|:------:|:------:|:------:|:------:|:------:|:------:|
|11|pomme|fruit	|sachet 1 kg	|1	|du 1/7/2022 au 31/12/2022|oui|
|12|pomme|fruit	|carton 10 kg	|9|du 1/7/2022 au 31/12/2022|oui|
|13|orange|fruit	|sachet 1 kg|2|du 1/7/2022 au 31/12/2022|fin 2022|
|14|orange|fruit	|carton 10 kg	|18	|du 1/7/2022 au 31/12/2022|fin 2022|
|15|piment|légume	|sachet 1 kg	|1.5|du 1/7/2022 au 31/12/2022|oui|
|16|piment|légume	|carton 10 kg|13|du 1/7/2022 au 31/12/2022|fin 2022|
|17|banane|fruit	|sachet 1 kg	|0.5|du 1/7/2022 au 31/12/2022|oui|
|18|banane|fruit	|carton 10 kg	|4|du 1/7/2022 au 31/12/2022|oui|
>
>*Il s'agit d'une liste de prix de différents aliments en fonction d'un conditionnement pour l'année 2022.*
>
> *On retrouve ici :*
> - *la classification : entre les produits et les aliments,* 
> - *le croisement : entre les produits et les quantités,*
> - *la caractérisation : la disponibilité du produit*

On peut noter que les deux usages de classification et de croisement conduisent à dupliquer fortement les informations avec les risques d'erreurs associés.

## Analyse des relations entre champs

Les relations entre deux champs peuvent être qualifiées et mesurées en fonction de ces trois usages. 
Ceci permet par exemple :
- de contrôler qu'une exigence est bien respectée,
- de retrouver les données qui ne respectent pas l'exigence 

> *Dans l'exemple précédent, on vérifie que le champ 'aliment' est bien 'derivé' du champ 'produit' ([voir partie 2 : lien entre deux colonnes](#lien-entre-deux-colonnes)).*
>
> *Si l'on introduit une association ('banane', 'légume') au lieu de ('banane', 'fruit'), l'exigence de couplage n'est plus respectée [(voir partie 2: erreur de saisie)](#erreur-de-saisie). Les données incohérentes peuvent alors être explicitée [(voir partie 2: détection des erreurs)](#détection-des-erreurs).*

Une des principales utilisation de ces relations entre champs est de pouvoir assurer une continuité entre :
- la définition initiale de la structure tabulaire au travers du modèle de données, 
- la conception de la structure par le schéma de données
- la validation de la documentation de la structure au fil de l'eau au a posteriori.    
    
La méthodologie de construction et de validation est décrite dans [un article spécifique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb). Elle permet de réduire significativement les erreurs de saisie et de documentation. 

## Analyse globale

L'analyse de l'ensemble des liens entre champs permet de représenter la structure globale du tableau.    
Une première représentation exploite le lien de classification.     
Elle se traduit par un arbre de dépendance entre colonnes.     


> *La première représentation appliquée à l'exemple se traduit par l'arborescence suivante ([voir partie 2: représentation globale](#représentation-globale)) :*
> ```python
> -1: root (8)    
>      0 : id (8)    
>            1 : produit (4)    
>                  2 : aliment (2)    
>            3 : quantite (2)
>            4 : prix (8)
>            6 : disponibilite (2)
>      5 : validite (1)
> ```
> *La racine de cette arborescence (root) représente la liste des lignes du tableaux.*    
> *Le champ 'aliment' est bien 'dérivé' du champ 'produit'.*    
> *la valeur entre parenthèse représente le nombre de valeurs différentes (les valeurs les plus faibles sont situées dans les feuilles de l'arbre).*
>

Une deuxième représentation qualifie chacun des champs en fonction de son rôle dans la structure globale (les catégories indiquées dans l'exemple ci-dessous sont précisée dans un article spécifique).
    
> *La deuxième représentation est la suivante ([voir partie 2: représentation globale](#représentation-globale)) :*
> ```python
> {'id': 'variable',
> 'produit': 'primary',
> 'aliment': 'secondary',
> 'quantite': 'primary',
> 'prix': 'coupled',
> 'validite': 'unique',
> 'disponibilite': 'variable'}
> ```
> *On distingue les catégories suivantes :*
> - *primary: qualifie les champs 'croisés' ('produit' et 'quantite')*
> - *secondary: représente les champs 'dérivés' des champs 'primary' ('aliment' qui est dérivé de 'produit')*
> - *coupled: est une catégorie spécique indiquant que chaque valeur de ce champ est associée à une unique valeur d'un autre champ (chaque valeur du champ 'prix' correspond à une valeur du champ 'id')*
> - *unique: pour les champs composés d'une seule valeur ('validite')
> - *variable: pour les autres champs qui ne disposent pas de relations particulières ('id', 'disponibilite')*

## Export vers des outils d'analyse
Certains outils exploitent nativement ces structures de données. C'est le cas par exemple de l'outil Xarray qui représente les données à la fois sous forme matricielle et sous forme de dépendances avec chacuns des axes de la matrice.

> *L'export des données de l'exemple vers Xarray produit la structure suivante ([voir partie 2: Export Xarray](#Export-Xarray))  :*
> ```python
><xarray.DataArray 'prix' (produit: 4, quantite: 2)>
>array([[20, 2],
>       [15, 1.5],
>       [5, 0.5],
>       [10, 1]], dtype=object)
>Coordinates:
>  * produit   (produit) object 'orange' 'piment' 'banane' 'pomme'
>  * quantite  (quantite) object 'carton 10 kg' 'sachet 1 kg'
>    aliment   (produit) object 'fruit' 'legume' 'fruit' 'fruit'
>Attributes:
>    validite:  du 1/7/2022 au 31/12/2022
> ```
> *Les données dans la matrice sont celles liées au champ 'prix', on aurait pu également choisir celles d'un autre champ 'variable' ('id' ou 'disponibilite')*
> *Les deux dimensions de la matrice correspondent aux deux champs 'produit' et 'quantite'.*
> *Le champ 'aliment' est associé au champ 'produit' au niveau de la structure 'coordinates'.*
> *Le champ 'annee' est unique et se trouve donc associé à la structure 'attributes'.*

## Optimisation des formats d'échange
Le format privilégié d'échange des données tabulaires est le format csv qui est l'image exacte d'un tableau (chaque ligne du tableau est une ligne du fichier csv).   
Ce format n'est pas optimal dans la mesure où les données dupliquées du tableau se retrouve également dupliquées dans le fichier csv.   
On peut également être confronté à des cas où l'objet reconstitué à partir des données csv n'est pas identique à l'objet d'origine.    
    
Ainsi, la prise en compte des liens entre champs permet de supprimer toutes les duplications et de les remplacer par une codification optimisée. Le gain de volume est alors conséquent puisque la taille du fichier est divisée par 5 à 10 sur des jeux de données importants.  
Cette transformation est également totalement réversible (l'objet reconstruit est identique à l'original).    

> *Les données ci-dessous représentent au format JSON les données de l'exemple dans une structure équivalente à celle d'un fichier csv ([voir partie 2: Format d'échange](#Format-d'échange)) :*      
>
>```python
>[['produit', ['piment', 'piment', 'banane', 'banane', 'pomme', 'pomme', 'orange', 'orange']],
 ['quantite', ['sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg']],
 ['aliment', ['legume', 'legume', 'fruit', 'fruit', 'fruit', 'fruit', 'fruit', 'fruit']],
 ['validite', ['du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022']],
 ['id', [15, 16, 17, 18, 11, 12, 13, 14]],
 ['disponibilite', ['oui', 'fin 2022', 'oui', 'oui', 'oui', 'oui', 'fin 2022', 'fin 2022']],
 ['prix', [1.5, 15, 0.5, 5, 1, 10, 2, 20]]]
>
>```
> *Si maintenant, on représente les mêmes informations mais sans duplication et avec une codification optimisée, on obtient un format JSON beaucoup plus simple ([voir partie 2: Format d'échange](#Format-d'échange)) :*      
>```python
> [['produit', ['piment', 'banane', 'pomme', 'orange']],
> ['quantite', ['sachet 1 kg', 'carton 10 kg']],
> ['aliment', ['fruit', 'legume'], [0, [1, 0, 0, 0]]],
> ['validite', ['du 1/7/2022 au 31/12/2022']],
> ['id', [15, 16, 17, 18, 11, 12, 13, 14]],
> ['disponibilite', ['fin 2022', 'oui'], [4, [1, 0, 1, 1, 1, 1, 0, 0]]],
> ['prix', [1.5, 15, 0.5, 5, 1, 10, 2, 20], 4]]
>```
> *Ce format JSON peut également être partagé dans un format binaire (standard CBOR) avec un gain supplémentaire de volume de données.*

## Identifiant unique

Disposer d'un identifiant unique pour une structure de données est un pré-requis pour garantir l'intégrité d'un jeu de données et assurer la traçabilité des modifications.
Cette notion est complexe et ne peut être traitée au niveau de la représentation textuelle ou physique d'une donnée. Elle doit nécessairement s'appliquer au niveau sémantique de la donnée.

>*Par exemple, les nombres réels $1.1$, et $0.11 \times 10^1$ doivent être considérés comme identiques tout comme deux dates doivent être considérées comme identiques si leur valeur dans le référentiel UTC sont identiques.*

Pour une structure de données tabulaire, l'identifiant doit également intégrer (en plus de l'identification de chacune des données qu'il contient) l'identifiant de la structure tabulaire qui porte les données.

>*Par exemple, le lien entre une données et la ligne ou la colonne qui la contient doit être pris en compte contrairement à l'ordre des colonnes qui n'est pas significatif.*

>*Ainsi, dans l'exemple présenté, les données reconstruites à partir du fichier JSON complet ou bien à partir du fichier JSON optimisé ou encore à partir du fichier binaire présentent le même identifiant ([voir partie 2: identifiant](#identifiant)).*


## Agrégation de structure

Les structures tabulaires ne sont pas restreintes à des données élémentaires. On peut, de façon récursive, assembler plusieurs tableaux au sein d'un même tableau.    

Ce tableau peut être 'déplié' pour faire apparaître les données élémentaires de chacun des tableaux inclus. Réciproquement, un tableau peut être 'replié' pour faire apparaître les données agrégées.

Cet usage couplé à l'utilisation d'identifiants permet de garantir et de vérifier que les données d'origine ont bien été respectées dans les assemblages successifs de données. Il facilite également les mises à jour puisqu'une modification de données d'entrée peut se propager automatiquemant aux données de sortie.

> *On peut par exemple considérer que le tableau de liste de prix précédent a été construit en assemblant un premier tableau relatif aux fruits :*    
>    
>|id|produit|quantité	|prix|disponibilité|
|:------:|:------:|:------:|:------:|:------:|
|11|pomme|sachet 1 kg	|1	|oui|
|12|pomme|carton 10 kg	|9|oui|
|13|orange|sachet 1 kg|2|fin 2022|
|14|orange|carton 10 kg	|18	|fin 2022|
|17|banane|sachet 1 kg	|0.5|oui|
|18|banane|carton 10 kg	|4|oui|
>    
> *et un second relatif aux légumes :*     
>
>|id|produit|quantité	|prix|disponibilité|
|:------:|:------:|:------:|:------:|:------:|
|15|piment|sachet 1 kg	|1.5|oui|
|16|piment|carton 10 kg|13|fin 2022|
> 
> *Le tableau de synthèse agrège les deux tableaux en ajoutant des champs supplémentaires :*     
>
>|aliment|validité|total|
|:------:|:------:|:------:|
|fruit|du 1/7/2022 au 31/12/2022|tableau 1|
|legume|du 1/7/2022 au 31/12/2022|tableau 2|
>
> *Ce tableau de synthèse 'déplié' est identique au tableau présenté en tête d'article ([voir partie 2: agrégation](#agrégation)).*

## Indicateurs qualité



## Moyen d'accès optimisé

Dans une structure tabulaire, les données sont dissociables : on peut accéder à l'ensemble des données ou bien à un ensemble de données réduit à quelques lignes ou quelques colonnes.   

Par contre, un accès sélectif nécessite au préalable soit un stockage en base de données, soit de charger le jeu de données dans un outil qui permet d'effectuer cette sélection.    
    
L'usage d'un format JSON supprime cette contrainte. En effet, celui-ci peut être stocké indifféremment dans un fichier ou dans une base de données "document". 

Par ailleurs, l'utilisation d'un format JSON standardisé permet également de standardiser les modes et les critères de sélection des données.

Cette approche est simple à mettre en oeuvre et répond au besoin de réduire les volumes de données échangés (on ne transfert que le juste nécessaire) tout en permettant de disposer de données répondant directement à la demande.

> *Le format JSON présenté ci-dessus est applicable pour tout type de données tabulaires. Le stockage du tableau de liste de prix dans une base de données 'document' (ex. MongoDB) ne nécessite donc aucun paramétrage particulier de la base. Dans l'exemple d'implémentation [présenté en partie 2](#accès-banalisé)), le chargement des données se résume à une ligne de code ou de commande*
>      
> *La structure du fichier json étant elle aussi standardisée, un éditeur de requête lui aussi standardisé peut être mis en place. Dans l'exemple d'implémentation [présentée en partie 2](#accès-banalisé), l'extraction du tableau de prix réduit uniquement aux 'légumes' est obtenue en fournissant uniquement le nom du champ concerné : 'aliment' ainsi que le nom de la valeur recherchée : 'légume'.*

------------------

# 2 - Exemple d'outil

Cette deuxième partie vient en appui de l'exemple présenté en partie 1.

Elle présente au travers d'une implémentation opérationnelle une validation des assertions de la partie 1.

### lien entre deux colonnes
(voir la partie 1 : [Analyse des relations entre champs](#Analyse-des-relations-entre-champs))    
     
L'objet 'tarif' est construit à partir des données indiquées.    
On vérifie que la règle de dépendance entre les deux champs est bien active.

In [1]:
from observation import Ilist
#création de l'exemple
tarif = Ilist.obj([['id', [11, 12, 13, 14, 15, 16, 17, 18]],
                   ['produit', ['pomme', 'pomme', 'orange', 'orange', 'piment', 'piment', 'banane', 'banane']],
                   ['aliment', ['fruit', 'fruit', 'fruit', 'fruit', 'legume', 'legume', 'fruit', 'fruit']],
                   ['quantite', ['sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg']],
                   ['prix', [1, 10, 2, 20, 1.5, 15, 0.5, 5]],
                   ['validite', ['du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022']],
                   ['disponibilite', ['oui', 'oui','fin 2022','fin 2022','oui','fin 2022','oui','oui']]])

id, produit, aliment, quantite, prix, validite, dispo = tarif.lindex

#test du lien entre le champ 'aliment' et le champ 'produit'
aliment.isderived(produit)

True

### erreur de saisie
(voir la partie 1 : [analyse des colonnes](#Analyse-des-colonnes))
    
L'exemple est reconstruit avec une erreur sur la septième valeur du champ 'aliment' ('legume' au lieu de 'fruit').    
La relation entre 'aliment' et 'produit' n'est maintenant plus active.

In [2]:
# création de l'exemple erroné
tarif_erreur = Ilist.obj([['id', [11, 12, 13, 14, 15, 16, 17, 18]],
                   ['produit', ['pomme', 'pomme', 'orange', 'orange', 'piment', 'piment', 'banane', 'banane']],
                   ['aliment', ['fruit', 'fruit', 'fruit', 'fruit', 'legume', 'legume', 'legume', 'fruit']],
                   ['quantite', ['sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg']],
                   ['prix', [1, 10, 2, 20, 1.5, 15, 0.5, 5]],
                   ['validite', ['du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022']],
                   ['disponibilite', ['oui', 'oui','fin 2022','fin 2022','oui','fin 2022','oui','oui']]])

id_, produit_, aliment_, quantite_, prix_, validite_, dispo_ = tarif_erreur.lindex

# test de la relation entre les deux champs
aliment_.isderived(produit_)

False

### détection des erreurs
(voir la partie 1 : [analyse des colonnes](#Analyse-des-colonnes))
     
La première instruction impose le couplage des champs 'produit' et 'aliment'.    
La seconde présente les enregistrements ne respectant pas le couplage imposé.

In [3]:
# forcage du couplage
produit_.coupling(aliment_)

# restitution des données non conformes
tarif_erreur.getduplicates(['produit'], indexview=['produit', 'aliment'] )

[('banane', 'legume'), ('banane', 'fruit')]

### représentation globale
(voir la partie 1 : [analyse globale](#Analyse-globale))

In [4]:
# représentation arborescente des liens de 'dérivation'
print(tarif.tree)

# affichage des catégories de chacun des champs
tarif.category

-1: root (8)
      0 : id (8)
            1 : produit (4)
                  2 : aliment (2)
            3 : quantite (2)
            4 : prix (8)
            6 : disponibilite (2)
      5 : validite (1)


{'id': 'variable',
 'produit': 'primary',
 'aliment': 'secondary',
 'quantite': 'primary',
 'prix': 'coupled',
 'validite': 'unique',
 'disponibilite': 'variable'}

### Export Xarray
(voir partie 1 : [Export vers des outils d'analyse](#Export-vers-des-outils-d'analyse))
     
Deux exemples sont donnés avec deux champs différents : 'prix' et 'disponibilite'

In [5]:
tarif.to_xarray(varname='prix')

<xarray.DataArray 'prix' (produit: 4, quantite: 2)>
array([[20, 2],
       [10, 1],
       [5, 0.5],
       [15, 1.5]], dtype=object)
Coordinates:
  * produit   (produit) object 'orange' 'pomme' 'banane' 'piment'
  * quantite  (quantite) object 'carton 10 kg' 'sachet 1 kg'
    aliment   (produit) object 'fruit' 'fruit' 'fruit' 'legume'
Attributes:
    validite:  du 1/7/2022 au 31/12/2022

In [6]:
tarif.to_xarray(varname='disponibilite')

<xarray.DataArray 'disponibilite' (produit: 4, quantite: 2)>
array([['fin 2022', 'fin 2022'],
       ['oui', 'oui'],
       ['oui', 'oui'],
       ['fin 2022', 'oui']], dtype=object)
Coordinates:
  * produit   (produit) object 'orange' 'pomme' 'banane' 'piment'
  * quantite  (quantite) object 'carton 10 kg' 'sachet 1 kg'
    aliment   (produit) object 'fruit' 'fruit' 'fruit' 'legume'
Attributes:
    validite:  du 1/7/2022 au 31/12/2022

### Format d'échange
(voir partie 1 : [Optimisation des formats d'échange](#Optimisation-des-formats-d'échange))    

Les trois formats ont des tailles différentes.    
Dans le format optimisé, aucune duplication n'est réalisée et les informations supplémentaires de codification sont très faibles :     
- [0, [0, 0, 1, 0]], 
- [4, [1, 1, 0, 0, 0, 1, 1, 1]]
- 4

In [64]:
from pprint import pprint

# arrangement de l'ordre des lignes et des colonnes
tarif.setcanonorder(reindex=True)

# génération du format JSON pour différentes options
fichier_csv = tarif.json(modecodec='full', encoded=True)
fichier_opt = tarif.json(modecodec='optimize', encoded=True)
fichier_bin = tarif.json(modecodec='optimize', encoded=True, encode_format='cbor')

# affichage de la taille (longueur) des formats texte ou binaires
print('taille : ', len(fichier_csv), len(fichier_opt), len(fichier_bin))

# affichage des deux formats texte
print('\nmode équivalent csv :')
pprint(tarif.json(modecodec='full'), width=250)
print('\nmode optimisé :')
pprint(tarif.json(modecodec='optimize'))

taille :  747 358 215

mode équivalent csv :
[['produit', ['banane', 'banane', 'orange', 'orange', 'piment', 'piment', 'pomme', 'pomme']],
 ['quantite', ['carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg']],
 ['aliment', ['fruit', 'fruit', 'fruit', 'fruit', 'legume', 'legume', 'fruit', 'fruit']],
 ['id', [18, 17, 14, 13, 16, 15, 12, 11]],
 ['disponibilite', ['oui', 'oui', 'fin 2022', 'fin 2022', 'fin 2022', 'oui', 'oui', 'oui']],
 ['prix', [5, 0.5, 20, 2, 15, 1.5, 10, 1]],
 ['validite', ['du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022']]]

mode optimisé :
[['produit', ['banane', 'orange', 'piment', 'pomme']],
 ['quantite', ['carton 10 kg', 'sachet 1 kg']],
 ['aliment', ['fruit', 'legume'], [0, [0, 0, 1, 0]]],
 ['id', [18, 17, 14, 13, 16

### identifiant
(voir partie 1 : [Identifiant unique](#Identifiant-unique))    
    
La fonction hash fournit l'identifiant unique de l'objet.   
Les objets reconstruits à partir de chacun des trois formats sont identiques.

In [8]:
# affichage de la fonction hash appliquée à chacun des objets reconstruits
print(hash(Ilist.from_obj(fichier_csv)))
print(hash(Ilist.from_obj(fichier_opt)))
print(hash(Ilist.from_obj(fichier_bin)))

389226330178274704
389226330178274704
389226330178274704


### agrégation
(voir partie 1 : [Agrégation de structure](#Agrégation-de-structure))    
    

In [9]:
fruit = Ilist.obj([['id', [11, 12, 13, 14, 17, 18]],
                   ['produit', ['pomme', 'pomme', 'orange', 'orange', 'banane', 'banane']],
                   ['quantite', ['sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg']],
                   ['prix', [1, 10, 2, 20, 0.5, 5]],
                   ['disponibilite', ['oui', 'oui','fin 2022','fin 2022','oui','oui']]])
legume = Ilist.obj([['id', [15, 16]],
                   ['produit', ['piment', 'piment']],
                   ['quantite', ['sachet 1 kg', 'carton 10 kg']],
                   ['prix', [1.5, 15]],
                   ['disponibilite', ['oui','fin 2022']]])

# L'objet 'cost' est l'agrégation de l'objet 'fruit' et l'objet 'legume'
cost = Ilist.obj([['aliment', ['fruit', 'legume']],
                  ['validite', ['du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022']],
                  ['total', [fruit, legume]]])

# L'objet cost_merged correspond à l'objet 'cost' déplié.
cost_merged = cost.merge(reindex=True)
cost_merged.setcanonorder(reindex=True)

# on retrouve bien l'objet 'tarif' initial
tarif == cost_merged

True

### accès banalisé
(voir partie 1 : [Moyen d'accès optimisé](#Moyen-d'accès-optimisé))    

In [56]:
from test_mongo import clientMongo
from essearch import ESSearch
from observation import Observation

collec = clientMongo()['test_obs']['observation']

# envoi de l'objet 'tarif' (l'objet Observation ajoute des méta-donnnées à l'objet Ilist) dans une base mongoDB
json_tarif = Observation(tarif, name='test_tarif').json(modecodec='dict')
print(collec.insert_one(json_tarif).inserted_id)

# définition du critère de recheche de l'objet 'tarif'
critere_tarif = {'path':'name', 'operand': 'test_tarif', 'comparator': '=='}

# vérification que l'objet récupéré (tarif_complet) est bien identique à celui envoyé
tarif_complet = ESSearch(critere_tarif, collec).execute(single=True)
print(Ilist(tarif_complet) == tarif)

#ajout d'un critère supplémentaire pour ne cherche que la partie de 'tarif' qui concerne les aliments de type 'legume'
tarif_legume = ESSearch([['aliment', 'legume'], critere_tarif], collec).execute(single=True)
print(Ilist(tarif_legume))


63924dafdb89f82121625e6a
False

    ["produit", ["piment", "piment", "piment", "piment"]]
    ["id", [16, 15, 16, 15]]
    ["aliment", ["legume", "legume", "legume", "legume"]]
    ["quantite", ["carton 10 kg", "sachet 1 kg", "carton 10 kg", "sachet 1 kg"]]
    ["prix", [15, 1.5, 15, 1.5]]
    ["disponibilite", ["fin 2022", "oui", "fin 2022", "oui"]]
    ["validite", ["du 1/7/2022 au 31/12/2022", "du 1/7/2022 au 31/12/2022", "du 1/7/2022 au 31/12/2022", "du 1/7/2022 au 31/12/2022"]]

